In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0
import sys; sys.path.append('.')

env: CUDA_VISIBLE_DEVICES=0


In [2]:
#TMP - rm when format gets fixed
_='\n'.join(filter(len, open("./data/bpe_parallel2.txt").read().split('\n')))
open("./data/bpe_parallel2.txt", 'w').write(_)

5453204

In [3]:
import numpy as np
from vocab import Vocab
inp_voc = Vocab.from_file("./data/1.voc")
out_voc = Vocab.from_file("./data/2.voc")

In [4]:
import tensorflow as tf
import models.transformer as tr
from lib.tensor_utils import infer_length, infer_mask

tf.reset_default_graph()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

mini_hp = {
        'hid_size': 256,
        'ff_size': 1024,
        'num_heads': 4,
        'num_layers': 2,
    
        'rescale_emb': True,
        'relu_dropout': 0.,
        'res_dropout': 0.,
        'attn_dropout': 0.,
        'inp_emb_bias': True,
        'res_steps': 'nlda',
        'normalize_out': True,
        'force_bos': False
}

model = tr.Transformer('mod',inp_voc,out_voc,**mini_hp)

_dummy = np.arange(5,30, dtype='int32').reshape([5,5])
inp = tf.placeholder_with_default(_dummy, [None,None])
out = tf.placeholder_with_default(_dummy, [None,None])
from lib.inference import BeamSearchInference
BeamSearchInference(model, {'inp':inp})

/home/jheuristic/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/jheuristic/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


/home/jheuristic/PycharmProjects/unsupervised_mt/babelSolution/lib/sliced_argmax_cpu.so


NotFoundError: /home/jheuristic/PycharmProjects/unsupervised_mt/babelSolution/lib/sliced_argmax_cpu.so: undefined symbol: _ZTIN10tensorflow8OpKernelE

In [ ]:
import tfnn
tfnn.__file__
import tensorflow as tf
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


In [ ]:
import tensorflow as tf
import models.transformer as tr
from lib.tensor_utils import infer_length, infer_mask

tf.reset_default_graph()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

mini_hp = {
        'hid_size': 256,
        'ff_size': 1024,
        'num_heads': 4,
        'num_layers': 2,
    
        'rescale_emb': True,
        'relu_dropout': 0.,
        'res_dropout': 0.,
        'attn_dropout': 0.,
        'inp_emb_bias': True,
        'res_steps': 'nlda',
        'normalize_out': True,
        'force_bos': False
}

model = tr.Transformer('mod',inp_voc,out_voc,**mini_hp)

In [ ]:
from prefetch_generator import background
from itertools import islice, chain
from tqdm import tqdm
class train:
    #TODO use batch_iterator or merge with it
    #TODO port length-aware batch iterator
    @background(max_prefetch=100)
    def iterate_lines(batch_size=16, batches_per_epoch=None, skip_lines=0,
                      src_path = "./data/bpe_parallel1.txt",
                      dst_path = "./data/bpe_parallel2.txt",
                      ):
        while True:
            with open(src_path) as f_src, open(dst_path) as f_dst:
                
                num_lines = batches_per_epoch * batch_size if batches_per_epoch is not None else None
                
                f_src = islice(f_src, skip_lines, num_lines)
                f_dst = islice(f_dst, skip_lines, num_lines)

                batch = []
                for src_line, dst_line in zip(f_src, f_dst):
                    if len(src_line) ==0 or len(dst_line)==0: continue
                    
                    batch.append([src_line[:-1], dst_line[:-1]])
                    if len(batch) >= batch_size:
                        yield (batch)
                        batch = []
                
                if batches_per_epoch is not None:
                    raise StopIteration()        
    
class dev:
    dev_src = "./data/bpe_parallel1.txt"
    dev_dst = "./data/bpe_parallel2.txt"

    inp_lines = np.array(open(dev_src).read()[:-1].split('\n'))
    out_lines = np.array(open(dev_dst).read()[:-1].split('\n'))

In [ ]:
from bleu import compute_bleu

In [ ]:
_dummy = np.arange(5,30, dtype='int32').reshape([5,5])
inp = tf.placeholder_with_default(_dummy, [None,None])
out = tf.placeholder_with_default(_dummy, [None,None])

In [ ]:
from lib.inference import BeamSearchInference
BeamSearchInference(model, {'inp':inp})

In [ ]:
logits = model.symbolic_score(inp, out, is_train=True)
# logits are already alligned with out

In [ ]:
from lib.tensor_utils import infer_mask, initialize_uninitialized_variables

nll = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=out)

loss = nll * infer_mask(out, out_voc.eos, dtype=tf.float32)

loss = tf.reduce_sum(loss, axis=1)
loss = tf.reduce_mean(loss)

In [ ]:
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, model.name)
grads = tf.gradients(loss, weights)
grads = tf.clip_by_global_norm(grads, 100)[0]
train_step = tf.train.AdamOptimizer(learning_rate=1e-4).apply_gradients(zip(grads,weights))

In [ ]:
initialize_uninitialized_variables(sess)

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import count
from pandas import ewma
from tqdm import tqdm
from time import sleep
epoch = 0
loss_history = []

In [ ]:
batch_size = 64
for i, batch in enumerate(tqdm(train.iterate_lines(batch_size, batches_per_epoch=None))):
    batch_src, batch_dst = zip(*batch)
    batch_src_ix = inp_voc.tokenize_many(batch_src)
    batch_dst_ix = out_voc.tokenize_many(batch_dst)
    
    feed_dict = {inp: batch_src_ix, out: batch_dst_ix}
    
    loss_t = sess.run([train_step, loss], feed_dict)[1]
    loss_history.append(np.mean(loss_t))
    
        
    if (epoch+1) % 500 == 0:
        #test_bleu_history.append([epoch,compute_bleu(model, dev.inp_lines[::5], dev.out_lines[::5])[0]])
        
        
        w_variables = tf.get_collection(tf.GraphKeys.MODEL_VARIABLES, scope="mod/")
        w_values = sess.run(w_variables)
        weights_dict = {w.name: w_val for w, w_val in zip(w_variables,w_values)}
        np.savez("/home/jheuristic/mimic-model-%i.npz"%epoch, **weights_dict)
        
    
    if epoch % 10 == 0:
        clear_output(True)
        plt.figure(figsize=[8,8])
        plt.title("batch loss")
        plt.plot(loss_history)
        plt.plot(ewma(np.array(loss_history),span=50))
        plt.grid()
        plt.show()
    epoch +=1
        

In [ ]:
from lib.inference import BeamSearchInference

In [ ]:
BeamSearchInference(model,{'inp':inp })